<a href="https://colab.research.google.com/github/aviax1/AE1/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**imports**

In [34]:
# used some sniped from https://github.com/L1aoXingyu/pytorch-beginner/blob/master/08-AutoEncoder/simple_autoencoder.py
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


**initial data+model setting**

In [37]:
batch_size = 128
#dataset = MNIST('./',download=True, ]))
#transform = transforms.Compose([ transforms.ToTensor(),transforms.Normalize([0.5], [0.5]))

(x,y), (_, _) = mnist.load_data()
dataloader = DataLoader(x, batch_size=batch_size, shuffle=True)
image_size=784
hidden_size=128
lv_size = 64  #Latent Variable 
learning_rate=1e-4
criterion = nn.MSELoss()
num_epochs=200
n=1

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    return x.view(x.size(0), 1, 28, 28)

**build model**

In [20]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, image_size),nn.ReLU(True), nn.Linear(image_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size), nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),nn.Linear(hidden_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

**train model by digit**

In [38]:
by_digit=0
model = autoencoder()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

image_count=0
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img=img[lebal==by_digit]
        img = Variable(img.view(img.size(0), -1))
        output = model(img)
        loss = criterion(output, img)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, num_epochs, loss.data))
    if 0: #epoch % 10 == 0:
        image_count+=1
        pic = to_img(output.cpu().data)
        nn,_,_,_ = np.shape(pic)
        print(np.shape(pic))
        for i in range(nn):
          ax = plt.subplot(2, n, image_count+i + 1)
          plt.imshow(pic[i,0,:,:].rehspae([28,28]))
          plt.title( "reconstruct image " )
          plt.gray()

torch.save(model.state_dict(), './sim_autoencoder.pth')

ValueError: ignored